## Define Function Decorators with functools.wraps

In [7]:
def trace(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print('%s(%r, %r) -> %r' % (func.__name__, args, kwargs, result))
        return result
    return wrapper


In [8]:
@trace
def fibonacci(n):
    ''' Return the n-th Fibonacci number '''
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))


In [9]:
fibonacci(3)

fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((3,), {}) -> 2


2

This works well, but it has an unintended side effect. The value returned by the decorator — the function that’s called above —  doesn’t think it's named fibonacci.

In [10]:
fibonacci

<function __main__.wrapper>

In [11]:
help(fibonacci)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



The solution is to use the wraps helper function from the functools built-in module.

In [15]:
import functools

def trace(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print('%s(%r, %r) -> %r' % (func.__name__, args, kwargs, result))
        return result
    return wrapper

@trace
def fibonacci(n):
    ''' Return the n-th Fibonacci number '''
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))


In [16]:
fibonacci(3)

fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((3,), {}) -> 2


2

In [17]:
fibonacci

<function __main__.fibonacci>

In [18]:
help(fibonacci)

Help on function fibonacci in module __main__:

fibonacci(*args, **kwargs)
    Return the n-th Fibonacci number



## Consider contextlib and with Statements for Reusable try/finally Behavior

In [26]:
# http://eigenhombre.com/introduction-to-context-managers-in-python.html
import contextlib
import time

@contextlib.contextmanager
def time_print(task_name):
    t = time.time()
    try:
        yield # wait for the task to complete
    finally:
        print task_name, "took", time.time() - t, "seconds."

with time_print("processes"):
    [x**2 for x in range(int(1e6))]


processes took 1.99599981308 seconds.


## Make pickle Reliable with copyreg

In [2]:
class GameState(object):
    def __init__(self):
        self.level = 0
        self.lives = 4
        
state = GameState()
state.level += 1
state.lives -= 1


In [3]:
import pickle

state_path = "game_state.bin"
with open(state_path, "wb") as f:
    pickle.dump(state, f)

In [5]:
# later we can load this back
with open(state_path, "rb") as f:
    state_after = pickle.load(f)
print(state_after.__dict__)

{'lives': 3, 'level': 1}


If you modify the GameState class to include more attributes the reading of this pickled data will still return an instate of GameState but with the old attributes (and not the new ones).

In [6]:
class GameState(object):
    def __init__(self):
        self.level = 0
        self.lives = 4
        self.points = 1

In [7]:
with open(state_path, "rb") as f:
    state_after = pickle.load(f)
print(state_after.__dict__)
print(type(state_after))

{'lives': 3, 'level': 1}
<class '__main__.GameState'>


In the simplest case we can use a constructor with default arguments and copyreg which allows us to register functions used when pickling objects:

In [13]:
class GameState(object):
    def __init__(self, level=0, lives=4, points=0):
        self.level = level
        self.lives = lives
        self.points = points

In [14]:
# https://docs.python.org/3/library/copyreg.html
def pickle_game_state(game_state):
    kwargs = game_state.__dict__
    return unpickle_game_state, (kwargs, )

def unpickle_game_state(kwargs):
    return GameState(**kwargs)


In [15]:
import six
if six.PY3:
    import copyreg # naming for python 3
else:
    import copy_reg as copyreg # and a workaround for 2

copyreg.pickle(GameState, pickle_game_state)

In [16]:
state = GameState()
state.points += 1000
serialized = pickle.dumps(state)
state_after = pickle.loads(serialized)
print(state_after.__dict__)

{'points': 1000, 'lives': 4, 'level': 0}


In [17]:
# now change the GameState definition
class GameState(object):
    def __init__(self, level=0, lives=4, points=0, magic=5):
        self.level = level
        self.lives = lives
        self.points = points
        self.magic = magic

In [18]:
state_after = pickle.loads(serialized)
print(state_after.__dict__) # gets the default value

{'magic': 5, 'points': 1000, 'lives': 4, 'level': 0}


There may be issues in loading unpickled objects where there have been changes to the GameState class that are not backwards compatible (for example, removing one of the constuctor default kwargs). In this case we can pass a version parameter to kwargs and control for old versions of the class. Other issues to control for include when the class name gets changed.

## Use built-in algorithms and data structures